In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dir_gams = "~/codes/CIAM-Work/gams-outputs/gams-results"

### RCP8.5 baseline results from Diaz 2016

file names

In [5]:
files_rcp85 = ["rcp85p50ref"+str(k)+".csv" for k in range(1,11)]

read the files, place in a list of Pandas DataFrames

In [46]:
dfG85_all = []
for file in files_rcp85:
    dfG85_all.append(pd.read_csv(dir_gams+"/rcp85/"+file))

concatenate all into a master file. don't worry about the times being out of order; the calculations will specify the year and type of damages to be summed up later

In [47]:
dfG85 = pd.concat(dfG85_all)

remove the list of DataFrames to save on some memory

In [53]:
del dfG85_all

In [55]:
dfG85["variable"].unique()

array(['FloodRetreat', 'RetreatCost', 'WetlandRetreat', 'RelocateRetreat',
       'StormCapitalRetreat', 'StormPopRetreat', 'Construct',
       'ProtectCost', 'WetlandProtect', 'StormCapitalProtect',
       'StormPopProtect', 'FloodNoAdapt', 'NoAdaptCost', 'WetlandNoAdapt',
       'RelocateNoAdapt', 'StormCapitalNoAdapt', 'StormPopNoAdapt',
       'FloodOptimalFixed', 'OptimalFixedCost', 'WetlandOptimalFixed',
       'RelocateOptimalFixed', 'StormCapitalOptimalFixed',
       'StormPopOptimalFixed', 'ConstructOptimalFixed'], dtype=object)

Gather up Diaz 2016 results into a DataFrame

In [180]:
len(dfG85.loc[(dfG85["time"]==1) & (dfG85["variable"]=="OptimalFixedCost"), "value"])

12046

In [182]:
NoAdaptCost, OptimalCost, FloodNoAdapt, WetlandNoAdapt, RelocateNoAdapt, StormCapitalNoAdapt, StormPopNoAdapt = [], [], [], [], [], [], []
Construct, ProtectCost, WetlandProtect, StormPopProtect, StormCapitalProtect = [], [], [], [], []
RetreatCost, WetlandRetreat, StormPopRetreat, StormCapitalRetreat, RelocateRetreat = [], [], [], [], []
for t in range(1,11):
    NoAdaptCost.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="NoAdaptCost"), "value"].sum())
    OptimalCost.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="OptimalFixedCost"), "value"].sum())
    FloodNoAdapt.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="FloodNoAdapt"), "value"].sum())
    WetlandNoAdapt.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="WetlandNoAdapt"), "value"].sum())
    RelocateNoAdapt.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="RelocateNoAdapt"), "value"].sum())
    StormCapitalNoAdapt.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="StormCapitalNoAdapt"), "value"].sum())
    StormPopNoAdapt.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="StormPopNoAdapt"), "value"].sum())
    Construct.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="ConstructOptimalFixed"), "value"].sum())
    ProtectCost.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="ProtectCost"), "value"].sum())
    WetlandProtect.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="WetlandProtect"), "value"].sum())
    StormPopProtect.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="StormPopProtect"), "value"].sum())
    StormCapitalProtect.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="StormCapitalProtect"), "value"].sum())
    RetreatCost.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="RetreatCost"), "value"].sum())
    WetlandRetreat.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="WetlandRetreat"), "value"].sum())
    StormPopRetreat.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="StormPopRetreat"), "value"].sum())
    StormCapitalRetreat.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="StormCapitalRetreat"), "value"].sum())
    RelocateRetreat.append(dfG85.loc[(dfG85["time"]==t) & (dfG85["variable"]=="RelocateRetreat"), "value"].sum())

dfDiaz = pd.DataFrame()
dfDiaz["time"] = list(range(2010,2110,10))
dfDiaz["NoAdapt"] = NoAdaptCost
dfDiaz["Optimal"] = OptimalCost
dfDiaz["FloodNoAdapt"] = FloodNoAdapt
dfDiaz["WetlandNoAdapt"] = WetlandNoAdapt
dfDiaz["RelocateNoAdapt"] = RelocateNoAdapt
dfDiaz["StormCapitalNoAdapt"] = StormCapitalNoAdapt
dfDiaz["StormPopNoAdapt"] = StormPopNoAdapt
dfDiaz["Construct"] = Construct
dfDiaz["ProtectCost"] = ProtectCost
dfDiaz["WetlandProtect"] = WetlandProtect
dfDiaz["StormPopProtect"] = StormPopProtect
dfDiaz["StormCapitalProtect"] = StormCapitalProtect
dfDiaz["RetreatCost"] = RetreatCost
dfDiaz["WetlandRetreat"] = WetlandRetreat
dfDiaz["StormPopRetreat"] = StormPopRetreat
dfDiaz["StormCapitalRetreat"] = StormCapitalRetreat
dfDiaz["RelocateRetreat"] = RelocateRetreat

Matches the paper well (Fig 2, right column), except in the last year when things jump up just a bit too much. Paper reports \\$2.2T in the NoAdapt case and \\$270B in the Optimal adaptation case.

In [145]:
print(np.round(dfDiaz[["time","NoAdapt","Optimal"]],2))

   time  NoAdapt  Optimal
0  2010   119.17    29.27
1  2020   189.34    33.03
2  2030   304.23    39.72
3  2040   484.77    52.75
4  2050   787.10   115.54
5  2060  1217.03   132.09
6  2070  1566.37   148.21
7  2080  1878.13   163.73
8  2090  2043.67   178.02
9  2100  2251.46   282.07


### New results

In [221]:
dfC = pd.read_csv("./output/lslOld_pop0_global_85p50ssp0fixed_control_sub1.csv")

In [270]:
def compare(dfD, dfN, var, lev=False, times=list(range(1,11)), verbose=True):
    varD = var
    names = {"OptimalCost" : "OptimalFixedCost"}#, "Construct" : "ConstructOptimalFixed"}
    if var in names.keys():
        varD = names[var]
    if lev:
        D = [dfD.loc[(dfD.variable==varD) & (dfD.level==lev) & (dfD.time==t), "value"].sum() for t in times]
        N = list(dfN.loc[(dfN.variable==var) & (dfN.level==lev) & (dfN.time>=times[0]) & (dfN.time<=np.max(times)), "value"])
    else:
        D = [dfD.loc[(dfD.variable==varD) & (dfD.time==t), "value"].sum() for t in times]
        N = list(dfN.loc[(dfN.variable==var) & (dfN.time>=times[0]) & (dfN.time<=np.max(times)), "value"])
    if verbose:
        print("time | Diaz | New")
        for t in range(len(times)):
            print(times[t], np.round(D[t],3), np.round(N[t],3))
    return [N[t]-D[t] for t in range(len(times))]

In [252]:
dfC["variable"].unique()

array(['FloodNoAdapt', 'RelocateNoAdapt', 'StormCapitalNoAdapt',
       'NoAdaptCost', 'OptimalCost', 'WetlandProtect', 'StormPopNoAdapt',
       'WetlandRetreat', 'WetlandNoAdapt', 'StormPopProtect',
       'StormCapitalProtect', 'ProtectCost', 'Construct',
       'StormCapitalRetreat', 'StormPopRetreat', 'RetreatCost',
       'FloodRetreat', 'RelocateRetreat'], dtype=object)

In [253]:
dfG85["variable"].unique()

array(['FloodRetreat', 'RetreatCost', 'WetlandRetreat', 'RelocateRetreat',
       'StormCapitalRetreat', 'StormPopRetreat', 'Construct',
       'ProtectCost', 'WetlandProtect', 'StormCapitalProtect',
       'StormPopProtect', 'FloodNoAdapt', 'NoAdaptCost', 'WetlandNoAdapt',
       'RelocateNoAdapt', 'StormCapitalNoAdapt', 'StormPopNoAdapt',
       'FloodOptimalFixed', 'OptimalFixedCost', 'WetlandOptimalFixed',
       'RelocateOptimalFixed', 'StormCapitalOptimalFixed',
       'StormPopOptimalFixed', 'ConstructOptimalFixed'], dtype=object)

Checking NoAdaptCost...

In [255]:
x = compare(dfG85, dfC, var="NoAdaptCost", lev=False, times=list(range(1,12)), verbose=True)

time | Diaz | New
1 119.166 119.166
2 189.336 189.336
3 304.226 304.226
4 484.775 484.775
5 787.095 787.095
6 1217.032 1217.038
7 1566.374 1566.376
8 1878.132 1878.142
9 2043.672 2043.716
10 2251.458 2251.467
11 2635.751 2635.751


Good!

Checking OptimalCost...

In [258]:
x = compare(dfG85, dfC, var="OptimalCost", lev=False, times=list(range(1,12)), verbose=True)

time | Diaz | New
1 29.274 27.608
2 33.032 31.593
3 39.718 38.638
4 52.752 52.064
5 115.535 124.3
6 132.086 143.298
7 148.209 161.838
8 163.731 180.723
9 178.017 197.697
10 282.067 325.643
11 303.188 352.242


Not good!

Check RetreatCost...

In [259]:
x = compare(dfG85, dfC, var="RetreatCost", lev=10, times=list(range(1,12)), verbose=True)

time | Diaz | New
1 123.019 123.019
2 173.769 173.769
3 277.287 277.287
4 496.142 496.142
5 279.418 279.418
6 327.637 327.637
7 407.305 407.305
8 549.06 549.06
9 847.47 847.47
10 577.476 577.476
11 624.229 624.229


Good!

Check ProtectCost...

In [260]:
x = compare(dfG85, dfC, var="ProtectCost", lev=10, times=list(range(1,12)), verbose=True)

time | Diaz | New
1 831.335 825.021
2 1333.589 1327.242
3 2411.871 2405.524
4 4775.877 4769.533
5 1339.202 1324.868
6 2118.873 2104.537
7 3735.674 3721.331
8 7285.422 7271.073
9 15344.465 15330.11
10 2325.17 2313.425
11 3528.217 3516.475


Not so good!

ProtectCost is composed of Construct, WetlandProtect, StormCapitalProtect, StormPopProtect, and FloodProtect, so check each of those...

`v.ProtectCost[tj,m,i-1] = v.Construct[tj,m,i-1] + v.WetlandProtect[tj,m] + v.StormCapitalProtect[tj,m,i-1] + v.StormPopProtect[tj,m,i-1] + v.FloodProtect[tj,m]`

Construction costs are different:

In [271]:
x = compare(dfG85, dfC, var="Construct", lev=10, times=list(range(1,12)), verbose=True)

time | Diaz | New
1 177.491 171.4
2 177.491 171.4
3 177.491 171.4
4 177.491 171.4
5 300.823 286.659
6 300.823 286.659
7 300.823 286.659
8 300.823 286.659
9 300.823 286.659
10 521.828 510.093
11 521.828 510.093


WetlandProtect costs are different

In [263]:
x = compare(dfG85, dfC, var="WetlandProtect", lev=False, times=list(range(1,12)), verbose=True)

time | Diaz | New
1 325.712 175.012
2 375.706 200.91
3 426.139 227.592
4 478.768 255.678
5 529.705 282.592
6 579.427 308.787
7 628.825 334.713
8 675.232 359.065
9 716.833 380.945
10 760.034 403.693
11 803.046 426.419


StormPopProtect and StormCapitalProtect costs match!

In [265]:
x = compare(dfG85, dfC, var="StormPopProtect", lev=10, times=list(range(1,12)), verbose=True)

time | Diaz | New
1 271.926 271.83
2 536.552 536.443
3 1106.339 1106.231
4 2339.987 2339.88
5 395.768 395.695
6 763.818 763.744
7 1531.877 1531.8
8 3236.131 3236.051
9 7150.682 7150.6
10 662.754 662.75
11 1228.236 1228.232


In [267]:
x = compare(dfG85, dfC, var="StormCapitalProtect", lev=10, times=list(range(1,12)), verbose=True)

time | Diaz | New
1 206.906 206.779
2 418.636 418.489
3 900.449 900.302
4 2002.721 2002.575
5 360.019 359.921
6 745.446 745.347
7 1568.261 1568.159
8 3389.404 3389.298
9 7512.015 7511.906
10 736.895 736.889
11 1351.734 1351.73


FloodProtect isn't on the output files

In [188]:
dfNew = pd.DataFrame()
dfNew["time"] = list(range(2010,2110,10))
for var in ["NoAdaptCost", "OptimalCost", "Construct", "RelocateRetreat",
            "FloodNoAdapt", "RelocateNoAdapt", "StormCapitalNoAdapt", "StormPopNoAdapt", "WetlandNoAdapt",
            "ProtectCost", "WetlandProtect", "StormPopProtect", "StormCapitalProtect",
            "RetreatCost", "WetlandRetreat", "StormPopRetreat", "StormCapitalRetreat", "RelocateRetreat"]:
    dfNew[var] = list(dfC.loc[dfC["variable"]==var, "value"])

ValueError: Length of values does not match length of index

Check that these 5 match the NoAdapt total cost:

In [136]:
dfNew["FloodNoAdapt"]+dfNew["RelocateNoAdapt"]+dfNew["StormCapitalNoAdapt"]+dfNew["StormPopNoAdapt"]+dfNew["WetlandNoAdapt"]-dfNew["NoAdapt"]

0   -2.842171e-13
1    1.762146e-12
2    6.252776e-13
3   -3.126388e-12
4    5.115908e-12
5    1.136868e-12
6    3.410605e-12
7   -8.185452e-12
8   -7.503331e-12
9   -4.092726e-12
dtype: float64

In [159]:
dfG85["variable"].unique()

array(['FloodRetreat', 'RetreatCost', 'WetlandRetreat', 'RelocateRetreat',
       'StormCapitalRetreat', 'StormPopRetreat', 'Construct',
       'ProtectCost', 'WetlandProtect', 'StormCapitalProtect',
       'StormPopProtect', 'FloodNoAdapt', 'NoAdaptCost', 'WetlandNoAdapt',
       'RelocateNoAdapt', 'StormCapitalNoAdapt', 'StormPopNoAdapt',
       'FloodOptimalFixed', 'OptimalFixedCost', 'WetlandOptimalFixed',
       'RelocateOptimalFixed', 'StormCapitalOptimalFixed',
       'StormPopOptimalFixed', 'ConstructOptimalFixed'], dtype=object)

In [160]:
dfC["variable"].unique()

array(['FloodNoAdapt', 'RelocateNoAdapt', 'StormCapitalNoAdapt',
       'NoAdaptCost', 'OptimalCost', 'WetlandProtect', 'StormPopNoAdapt',
       'WetlandRetreat', 'WetlandNoAdapt', 'StormPopProtect',
       'StormCapitalProtect', 'ProtectCost', 'Construct',
       'StormCapitalRetreat', 'StormPopRetreat', 'RetreatCost',
       'FloodRetreat', 'RelocateRetreat'], dtype=object)

In [163]:
len(dfG85["segments"].unique())

12148

In [156]:
for col in dfNew.columns:
    print(col+"\nYear| Diaz | New CIAM | Diaz-New")
    for t in dfDiaz["time"]:
        print(t, np.round(float(dfDiaz.loc[dfDiaz["time"]==t, col]),3), np.round(float(dfNew.loc[dfNew["time"]==t, col]),3), 
              np.round(float(dfDiaz.loc[dfDiaz["time"]==t, col]-dfNew.loc[dfNew["time"]==t, col]),3))
    print("")

time
Year| Diaz | New CIAM | Diaz-New
2010 2010.0 2010.0 0.0
2020 2020.0 2020.0 0.0
2030 2030.0 2030.0 0.0
2040 2040.0 2040.0 0.0
2050 2050.0 2050.0 0.0
2060 2060.0 2060.0 0.0
2070 2070.0 2070.0 0.0
2080 2080.0 2080.0 0.0
2090 2090.0 2090.0 0.0
2100 2100.0 2100.0 0.0

NoAdapt
Year| Diaz | New CIAM | Diaz-New
2010 119.166 119.166 0.0
2020 189.336 189.336 -0.0
2030 304.226 304.226 -0.0
2040 484.775 484.775 0.0
2050 787.095 787.095 -0.0
2060 1217.032 1217.038 -0.005
2070 1566.374 1566.376 -0.002
2080 1878.132 1878.142 -0.01
2090 2043.672 2043.716 -0.044
2100 2251.458 2189.244 62.213

Optimal
Year| Diaz | New CIAM | Diaz-New
2010 29.274 27.608 1.666
2020 33.032 31.593 1.439
2030 39.718 38.638 1.08
2040 52.752 52.064 0.687
2050 115.535 124.3 -8.765
2060 132.086 143.298 -11.212
2070 148.209 161.838 -13.629
2080 163.731 180.723 -16.992
2090 178.017 197.697 -19.68
2100 282.067 289.047 -6.979

FloodNoAdapt
Year| Diaz | New CIAM | Diaz-New
2010 40.34 40.34 -0.0
2020 64.955 64.955 -0.0
2030 101.8

In [472]:
dfN2 = pd.read_csv("./output/lslOld_pop0_seg_85p50ssp0fixed_control_sub1.csv")
dfC2 = pd.read_csv("./output/lslOld_pop0_global_85p50ssp0fixed_control_sub1.csv")

In [449]:
#segments_test = ["UnitedKingdom8819", "Sweden9129", "Ireland8492", "UnitedStates3383", "Canada5106", "Denmark7923", "Denmark7037"]
dfS = pd.read_csv("./ciam-code/data/subsets/sub1names.csv", header=None)
segments_test = list(dfS[0])

In [450]:
dfG2 = dfG85.loc[dfG85["segments"].isin(segments_test),:]

In [468]:
x = compare(dfG2, dfC2, var="Construct", lev=10, times=list(range(1,12)), verbose=True)

time | Diaz | New
1 51.805 47.414
2 51.805 47.414
3 51.805 47.414
4 51.805 47.414
5 65.954 61.495
6 65.954 61.495
7 65.954 61.495
8 65.954 61.495
9 65.954 61.495
10 92.029 87.873
11 92.029 87.873


In [288]:
dfC2["variable"].unique()

array(['FloodNoAdapt', 'RelocateNoAdapt', 'StormCapitalNoAdapt',
       'NoAdaptCost', 'OptimalCost', 'WetlandProtect', 'StormPopNoAdapt',
       'WetlandRetreat', 'WetlandNoAdapt', 'StormPopProtect',
       'StormCapitalProtect', 'ProtectCost', 'Construct',
       'StormCapitalRetreat', 'StormPopRetreat', 'RetreatCost',
       'FloodRetreat', 'RelocateRetreat'], dtype=object)

In [289]:
dfG2["variable"].unique()

array(['FloodRetreat', 'RetreatCost', 'RelocateRetreat',
       'StormCapitalRetreat', 'StormPopRetreat', 'WetlandRetreat',
       'Construct', 'ProtectCost', 'StormCapitalProtect',
       'StormPopProtect', 'WetlandProtect', 'FloodNoAdapt', 'NoAdaptCost',
       'RelocateNoAdapt', 'StormCapitalNoAdapt', 'StormPopNoAdapt',
       'WetlandNoAdapt', 'FloodOptimalFixed', 'OptimalFixedCost',
       'RelocateOptimalFixed', 'StormCapitalOptimalFixed',
       'StormPopOptimalFixed', 'WetlandOptimalFixed'], dtype=object)

In [340]:
float(dfN2.loc[(dfN2.variable=="OptimalCost") & (dfN2.time==13) & (dfN2.segments=="Sweden9129"), "value"]) - \
float(dfG2.loc[(dfG2.variable=="OptimalFixedCost") & (dfG2.time==13) & (dfG2.segments=="Sweden9129"), "value"])

0.01624851563499108

In [376]:
float(dfN2.loc[(dfN2.variable=="RetreatCost") & (dfN2.time==13) & (dfN2.level==10) & (dfN2.segments==segments_test[0]), "value"])

0.0015456638775497278

In [446]:
tol = 0.0001
lev = 10
t = 1
for seg in segments_test:
#    minlevel = dfN2.loc[(dfN2.variable=="RetreatCost") & (dfN2.time==t) & (dfN2.segments==seg), "level"].min()
    N = float(dfN2.loc[(dfN2.variable=="Construct") & (dfN2.time==t) & (dfN2.level==lev) & (dfN2.segments==seg), "value"])
    D = float(dfG2.loc[(dfG2.variable=="Construct") & (dfG2.time==t) & (dfG2.level==lev) & (dfG2.segments==seg), "value"])
    if (np.abs(N-D) > tol):
        print(seg,N,D)
#    if minlevel > 1:
#        cnt += 1

Canada5527 0.004040049401722718 0.056436152
Canada5687 0.0005090303345894624 0.005378783000000001
Canada5691 0.0023110227253425727 0.033551626
Canada5698 7.45997455923003e-05 0.007842693000000001
Finland9131 0.002290062791981559 0.006076201
Finland9137 0.00013856395260966722 0.0006650289999999999
Finland9138 0.008031941393879784 0.029487435
Finland9139 0.00010182675006077281 0.000557137
Finland9141 0.002310969919072248 0.007019585
Norway8935 0.0 0.009717973000000001
Norway8936 0.0 0.009549451
Norway8944 0.0 0.05078472
Norway8946 0.0 0.308962127
Norway8947 0.0 0.005926501999999999
Norway8948 0.0 0.01753626
Norway8949 0.0 0.001768751
Norway8950 0.0 0.001450895
Norway8951 0.0 0.000693165
Norway8952 0.0 0.0066607540000000005
Norway8953 0.0 0.003687401
Norway8954 0.0 0.003861567
Norway8955 0.0 0.003183978
Norway8956 0.0 0.002105909
Norway8957 0.0 0.001047367
Norway8958 0.0 0.007419499999999999
Norway8959 0.0 0.001574155
Norway8960 0.0 0.024964677
Norway8961 6.63770647035457e-06 0.000523944


In [439]:
cnt

79

In [475]:
dfN2.head()

,time,regions,segments,level,variable,value
0,1,CAN,Canada3384,NaN,FloodNoAdapt,0.000074
1,1,CAN,Canada3452,NaN,FloodNoAdapt,0.000191
2,1,CAN,Canada5092,NaN,FloodNoAdapt,0.000090
3,1,CAN,Canada5100,NaN,FloodNoAdapt,0.000011
4,1,CAN,Canada5103,NaN,FloodNoAdapt,0.000006


In [476]:
dfN2.loc[(dfN2.segments=="Canada5527") & (dfN2.time==1) & (dfN2.variable=="Construct")]

,time,regions,segments,level,variable,value
480780,1,CAN,Canada5527,10.0,Construct,0.004040
500780,1,CAN,Canada5527,100.0,Construct,0.051246
520780,1,CAN,Canada5527,1000.0,Construct,0.085356
540780,1,CAN,Canada5527,10000.0,Construct,0.542185
560780,1,CAN,Canada5527,0.0,Construct,0.000000


In [478]:
dfG2.loc[(dfG2.segments=="Canada5527") & (dfG2.time==1) & (dfG2.variable=="Construct")]

,time,segments,level,variable,value
356658,1,Canada5527,10.0,Construct,0.056436
423418,1,Canada5527,100.0,Construct,0.058993
489078,1,Canada5527,1000.0,Construct,0.079407
554558,1,Canada5527,10000.0,Construct,0.352820


In [408]:
cnt = 0
for seg in segments_test:
    if (dfG2.loc[(dfG2.variable=="RetreatCost") & (dfG2.time==1) & (dfG2.segments==seg), "level"].min() > 1):
        cnt += 1

In [ ]:
cnt

In [440]:
seg1 = "Canada5527"

In [442]:
dfG85.loc[(dfG85.variable=="RetreatCost") & (dfG85.time==1) & (dfG85.segments==seg1)]

,time,segments,level,variable,value
139940,1,Canada5527,100.0,RetreatCost,0.000939
210252,1,Canada5527,1000.0,RetreatCost,0.002948
281298,1,Canada5527,10000.0,RetreatCost,0.015136


In [444]:
dfS.loc[(dfS.variable=="RetreatCost") & (dfS.time==1) & (dfS.segments==seg1)]

,time,regions,segments,level,variable,value
820780,1,CAN,Canada5527,1.0,RetreatCost,0.000000
840780,1,CAN,Canada5527,10.0,RetreatCost,0.000000
860780,1,CAN,Canada5527,100.0,RetreatCost,0.000939
880780,1,CAN,Canada5527,1000.0,RetreatCost,0.002948
900780,1,CAN,Canada5527,10000.0,RetreatCost,0.015136
920780,1,CAN,Canada5527,0.0,RetreatCost,0.000000


In [445]:
dfG85.loc[(dfG85.variable=="Construct") & (dfG85.time==1) & (dfG85.segments==seg1)]

,time,segments,level,variable,value
356658,1,Canada5527,10.0,Construct,0.056436
423418,1,Canada5527,100.0,Construct,0.058993
489078,1,Canada5527,1000.0,Construct,0.079407
554558,1,Canada5527,10000.0,Construct,0.352820


In [447]:
dfS.loc[(dfS.variable=="Construct") & (dfS.time==1) & (dfS.segments==seg1)]

,time,regions,segments,level,variable,value
480780,1,CAN,Canada5527,10.0,Construct,0.004040
500780,1,CAN,Canada5527,100.0,Construct,0.051246
520780,1,CAN,Canada5527,1000.0,Construct,0.085356
540780,1,CAN,Canada5527,10000.0,Construct,0.542185
560780,1,CAN,Canada5527,0.0,Construct,0.000000


### Checking all RetreatCost, or other quantities with `levels`, for a certain time

In [488]:
tol = 0.0001
bad_segs = []
var = "RetreatCost"
t = 1
for seg in segments_test:
    levels = dfG2.loc[(dfG2.time==t) & (dfG2.variable==var) & (dfG2.segments==seg), "level"].unique()
    D = [float(dfG2.loc[(dfG2.variable==var) & (dfG2.level==lev) & (dfG2.time==t) & (dfG2.segments==seg), "value"]) for lev in levels]
    N = [float(dfN2.loc[(dfN2.variable==var) & (dfN2.level==lev) & (dfN2.time==t) & (dfN2.segments==seg), "value"]) for lev in levels]
    max_err = np.max([N[i] - D[i] for i in range(len(levels))])
    if max_err > tol:
        bad_segs.append(seg)
        print(seg, max_err, "<========================")
#    else:
        #print(seg, max_err)

### Checking all ProtectCost

In [517]:
tol = 0.0001
bad_segs = []
var = "ProtectCost"
t = 1
for seg in segments_test:
    levels = dfG2.loc[(dfG2.time==t) & (dfG2.variable==var) & (dfG2.segments==seg), "level"].unique()
    D = [float(dfG2.loc[(dfG2.variable==var) & (dfG2.level==lev) & (dfG2.time==t) & (dfG2.segments==seg), "value"]) for lev in levels]
    N = [float(dfN2.loc[(dfN2.variable==var) & (dfN2.level==lev) & (dfN2.time==t) & (dfN2.segments==seg), "value"]) for lev in levels]
    max_err = np.max([N[i] - D[i] for i in range(len(levels))])
    if max_err > tol:
        bad_segs.append(seg)
        print(seg, max_err, "<========================")
#    else:
        #print(seg, max_err)

Canada5527 0.1893652968916057 <========================
Canada5687 0.018379222159555675 <========================
Canada5691 0.11234703253702003 <========================
Canada5698 0.024907150539546515 <========================
Norway8935 0.07640089147347512 <========================
Norway8936 0.07654756517568732 <========================
Norway8944 0.3998619855312493 <========================
Norway8946 2.383589149912153 <========================
Norway8947 0.047441195449225565 <========================
Norway8948 0.13590917641936595 <========================
Norway8949 0.013635878728116748 <========================
Norway8950 0.011377199866797347 <========================
Norway8951 0.00545711443805138 <========================
Norway8952 0.052758567206560844 <========================
Norway8953 0.028897040303193535 <========================
Norway8954 0.029853595454764688 <========================
Norway8955 0.024783529937449582 <========================
Norway8956 0.0164679430678

In [520]:
print(bad_segs)

['Canada5527', 'Canada5687', 'Canada5691', 'Canada5698', 'Norway8935', 'Norway8936', 'Norway8944', 'Norway8946', 'Norway8947', 'Norway8948', 'Norway8949', 'Norway8950', 'Norway8951', 'Norway8952', 'Norway8953', 'Norway8954', 'Norway8955', 'Norway8956', 'Norway8957', 'Norway8958', 'Norway8959', 'Norway8960', 'Norway8961', 'Norway8962', 'Norway8963', 'Norway8964', 'Norway8965', 'Norway8966', 'Norway8967', 'Norway8968', 'Norway8969', 'Norway8970', 'Norway8971', 'Norway8972', 'Norway8973', 'Norway8974', 'Norway8975', 'Norway9090', 'Norway9109', 'Norway9111', 'Norway9115', 'Norway9117', 'Norway9118', 'Norway9119', 'Norway9121', 'Norway9122', 'Norway9123', 'Norway9127', 'Norway9130', 'Norway9135', 'Norway9152', 'Norway9168', 'Norway9169', 'Norway9170', 'Norway9171', 'Norway9172', 'Norway9173', 'Norway9174', 'Norway9175', 'Norway9176', 'Norway9177', 'Norway9178', 'Norway9179', 'Norway9180', 'Norway9181', 'Norway9182', 'Norway9183', 'RussianFederation9285', 'RussianFederation9289', 'RussianFed

In [518]:
len(bad_segs)

183

ProtectCost is composed of:
`                                v.ProtectCost[tj,m,i-1] = v.Construct[tj,m,i-1] + v.WetlandProtect[tj,m] + v.StormCapitalProtect[tj,m,i-1] + v.StormPopProtect[tj,m,i-1] + v.FloodProtect[tj,m]
`

So check those

### Checking all WetlandProtect, or other quantities without `levels`, over all times

In [510]:
tol = 0.0001
bad_segs = []
var = "WetlandProtect"
segments_wetland = dfG2.loc[(dfG2.variable==var) & (dfG2.time==1), "segments"].unique()
for seg in segments_wetland:
    D = [float(dfG2.loc[(dfG2.variable==var) & (dfG2.time==t) & (dfG2.segments==seg), "value"].iloc[0]) for t in range(1,15)]
    N = [float(dfN2.loc[(dfN2.variable==var) & (dfN2.time==t) & (dfN2.segments==seg), "value"]) for t in range(1,15)]
    max_err = np.max([N[i] - D[i] for i in range(len(N))])
    if max_err > tol:
        bad_segs.append(seg)
        print(seg, max_err, "<========================")
#    else:
        #print(seg, max_err)

In [525]:
seg

'Canada5106'

In [528]:
dfG2.variable.unique()

array(['FloodRetreat', 'RetreatCost', 'WetlandRetreat', 'RelocateRetreat',
       'StormCapitalRetreat', 'StormPopRetreat', 'Construct',
       'ProtectCost', 'WetlandProtect', 'StormCapitalProtect',
       'StormPopProtect', 'FloodNoAdapt', 'NoAdaptCost', 'WetlandNoAdapt',
       'RelocateNoAdapt', 'StormCapitalNoAdapt', 'StormPopNoAdapt',
       'FloodOptimalFixed', 'OptimalFixedCost', 'WetlandOptimalFixed',
       'RelocateOptimalFixed', 'StormCapitalOptimalFixed',
       'StormPopOptimalFixed', 'ConstructOptimalFixed'], dtype=object)

In [549]:
tol = 0.0001
bad_segs = []
var = "StormCapitalProtect"
t = 1
for seg in segments_test:
    levels = dfG2.loc[(dfG2.time==t) & (dfG2.variable==var) & (dfG2.segments==seg), "level"].unique()
    if (len(levels) > 0):
        D = [float(dfG2.loc[(dfG2.variable==var) & (dfG2.level==lev) & (dfG2.time==t) & (dfG2.segments==seg), "value"]) for lev in levels]
        N = [float(dfN2.loc[(dfN2.variable==var) & (dfN2.level==lev) & (dfN2.time==t) & (dfN2.segments==seg), "value"]) for lev in levels]
        max_err = np.max([N[i] - D[i] for i in range(len(levels))])
        if max_err > tol:
            bad_segs.append(seg)
            print(seg, max_err, "<========================")
    else:
        N = list(dfN2.loc[(dfN2.variable==var) & (dfN2.time==t) & (dfN2.segments==seg), "value"])
        if np.abs(np.median(N)) > tol/10:
            bad_segs.append(seg)
            print(seg, N, "<============ no StormCapitalProtect value issue")

In [550]:
tol = 0.0001
bad_segs = []
var = "StormPopProtect"
t = 1
for seg in segments_test:
    levels = dfG2.loc[(dfG2.time==t) & (dfG2.variable==var) & (dfG2.segments==seg), "level"].unique()
    if (len(levels) > 0):
        D = [float(dfG2.loc[(dfG2.variable==var) & (dfG2.level==lev) & (dfG2.time==t) & (dfG2.segments==seg), "value"]) for lev in levels]
        N = [float(dfN2.loc[(dfN2.variable==var) & (dfN2.level==lev) & (dfN2.time==t) & (dfN2.segments==seg), "value"]) for lev in levels]
        max_err = np.max([N[i] - D[i] for i in range(len(levels))])
        if max_err > tol:
            bad_segs.append(seg)
            print(seg, max_err, "<========================")
    else:
        N = list(dfN2.loc[(dfN2.variable==var) & (dfN2.time==t) & (dfN2.segments==seg), "value"])
        if np.abs(np.median(N)) > tol/10:
            bad_segs.append(seg)
            print(seg, N, "<============ no StormPopProtect value issue")

In [553]:
dfG2.variable.unique()

array(['FloodRetreat', 'RetreatCost', 'WetlandRetreat', 'RelocateRetreat',
       'StormCapitalRetreat', 'StormPopRetreat', 'Construct',
       'ProtectCost', 'WetlandProtect', 'StormCapitalProtect',
       'StormPopProtect', 'FloodNoAdapt', 'NoAdaptCost', 'WetlandNoAdapt',
       'RelocateNoAdapt', 'StormCapitalNoAdapt', 'StormPopNoAdapt',
       'FloodOptimalFixed', 'OptimalFixedCost', 'WetlandOptimalFixed',
       'RelocateOptimalFixed', 'StormCapitalOptimalFixed',
       'StormPopOptimalFixed', 'ConstructOptimalFixed'], dtype=object)

In [555]:
dfN2.variable.unique()

array(['FloodNoAdapt', 'RelocateNoAdapt', 'StormCapitalNoAdapt',
       'NoAdaptCost', 'OptimalCost', 'WetlandProtect', 'StormPopNoAdapt',
       'WetlandRetreat', 'WetlandNoAdapt', 'StormPopProtect',
       'StormCapitalProtect', 'ProtectCost', 'Construct',
       'StormCapitalRetreat', 'StormPopRetreat', 'RetreatCost',
       'FloodRetreat', 'RelocateRetreat'], dtype=object)

In [552]:
dfG2.loc[dfG2.variable=="FloodProtect"]

,time,segments,level,variable,value


In [556]:
tol = 0.0001
bad_segs = []
var = "FloodRetreat"
t = 1
for seg in segments_test:
    levels = dfG2.loc[(dfG2.time==t) & (dfG2.variable==var) & (dfG2.segments==seg), "level"].unique()
    if (len(levels) > 0):
        D = [float(dfG2.loc[(dfG2.variable==var) & (dfG2.level==lev) & (dfG2.time==t) & (dfG2.segments==seg), "value"]) for lev in levels]
        N = [float(dfN2.loc[(dfN2.variable==var) & (dfN2.level==lev) & (dfN2.time==t) & (dfN2.segments==seg), "value"]) for lev in levels]
        max_err = np.max([N[i] - D[i] for i in range(len(levels))])
        if max_err > tol:
            bad_segs.append(seg)
            print(seg, max_err, "<========================")
    else:
        N = list(dfN2.loc[(dfN2.variable==var) & (dfN2.time==t) & (dfN2.segments==seg), "value"])
        if np.abs(np.median(N)) > tol/10:
            bad_segs.append(seg)
            print(seg, N, "<============ no FloodRetreat value issue")

In [515]:
print(dfG2.loc[(dfG2.variable==var) & (dfG2.time==t) & (dfG2.segments==seg)])
print(dfN2.loc[(dfN2.variable==var) & (dfN2.time==t) & (dfN2.segments==seg)])

        time    segments    level     variable     value
385541     1  Canada3384     10.0  ProtectCost  0.087623
452281     1  Canada3384    100.0  ProtectCost  0.133994
517761     1  Canada3384   1000.0  ProtectCost  0.167694
583241     1  Canada3384  10000.0  ProtectCost  0.592483
        time regions    segments    level     variable     value
380000     1     CAN  Canada3384     10.0  ProtectCost  0.087623
400000     1     CAN  Canada3384    100.0  ProtectCost  0.133994
420000     1     CAN  Canada3384   1000.0  ProtectCost  0.167694
440000     1     CAN  Canada3384  10000.0  ProtectCost  0.592483
460000     1     CAN  Canada3384      0.0  ProtectCost  0.000007
